<a href="https://colab.research.google.com/github/subandwho/tensorflow/blob/main/FoodVisonBig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-4512110d-e02e-9d19-69e8-f01925e63555)


In [10]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-07-18 08:39:12--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-18 08:39:12 (107 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [11]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

In [12]:
import tensorflow_datasets as tfds
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [13]:
(train_data, test_data), ds_info = tfds.load(name = 'food101',
                                             split=['train', 'validation'],
                                             shuffle_files = True, 
                                             as_supervised = True,
                                             with_info = True,
                                             )

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incomplete2AK6QB/food101-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incomplete2AK6QB/food101-validation.tfrecord


Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [14]:
ds_info.features


FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [15]:
class_names  = ds_info.features['label'].names

In [16]:
train_one_sample = train_data.take(1)
train_one_sample

<TakeDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [17]:
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image dtype: {image.dtype}
  Target class from Food101 (tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
        """)


  Image shape: (512, 512, 3)
  Image dtype: <dtype: 'uint8'>
  Target class from Food101 (tensor form): 25
  Class name (str form): club_sandwich
        


In [18]:
import tensorflow as tf
import matplotlib.pyplot as plt
def preprocess_img(image, label, img_shape = 224):
  image = tf.image.resize(image, [img_shape, img_shape])
  return tf.cast(image, tf.float32), label

In [19]:

# What are the min and max values?
tf.reduce_min(image), tf.reduce_max(image)

(<tf.Tensor: shape=(), dtype=uint8, numpy=0>,
 <tf.Tensor: shape=(), dtype=uint8, numpy=255>)

In [20]:
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before preprocessing:\n {image[:2]}...,\nShape: {image.shape},\nDatatype: {image.dtype}\n")
print(f"Image after preprocessing:\n {preprocessed_img[:2]}...,\nShape: {preprocessed_img.shape},\nDatatype: {preprocessed_img.dtype}")

Image before preprocessing:
 [[[135 156 175]
  [125 148 166]
  [114 136 159]
  ...
  [ 26   5  12]
  [ 26   3  11]
  [ 27   4  12]]

 [[128 150 171]
  [115 140 160]
  [102 127 149]
  ...
  [ 28   7  14]
  [ 29   6  14]
  [ 30   7  15]]]...,
Shape: (512, 512, 3),
Datatype: <dtype: 'uint8'>

Image after preprocessing:
 [[[122.83163   146.17346   165.81633  ]
  [ 95.07653   122.122444  144.47958  ]
  [ 72.5051    106.994896  134.34694  ]
  ...
  [ 20.714308    2.3570995   3.9285717]
  [ 27.285715    6.285714   13.285714 ]
  [ 28.28575     5.2857494  13.285749 ]]

 [[ 88.65305   119.41326   140.41327  ]
  [ 74.59694   108.30102   133.02042  ]
  [ 75.2551    112.57143   141.91325  ]
  ...
  [ 26.857143    6.285671   11.040798 ]
  [ 30.061235    6.86222    16.795908 ]
  [ 31.688843    5.688843   16.688843 ]]]...,
Shape: (224, 224, 3),
Datatype: <dtype: 'float32'>


In [21]:
train_data

<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [22]:
train_data = train_data.map(map_func = preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [23]:
train_data, test_data


(<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>,
 <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>)

In [24]:
from helper_functions import create_tensorboard_callback
checkpoint_path = "model_checkpoint/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy',
                                                      save_best_only = True,
                                                     save_weights_only = True ,
                                                      verbose=0)

In [25]:
tf.__version__

'2.4.1'

In [26]:
!pip install tensorflow==2.4.1

In [27]:
import tensorflow as tf
tf.__version__


'2.4.1'

In [30]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [31]:
from tensorflow.keras import layers
from tensorflow.keras import preprocessing

input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False
inputs = layers.Input(shape = input_shape, name = "Input_Layer")
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype = tf.float32, name = "softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = "Adam",
              metrics = ['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 101)               129381    
_________________________________________________________________
softmax_float32 (Activation) (None, 101)               0         
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [32]:
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype_policy, layer.dtype)

Input_Layer True <Policy "float32"> float32
efficientnetb0 False <Policy "mixed_float16"> float32
global_average_pooling2d_2 True <Policy "mixed_float16"> float32
dense_1 True <Policy "mixed_float16"> float32
softmax_float32 True <Policy "float32"> float32


In [34]:
history1 = model.fit(train_data, epochs = 3, validation_data = test_data,
                     validation_steps = len(test_data),
                     callbacks = [create_tensorboard_callback(dir_name='project1', experiment_name='softmax_101'),
                                    model_checkpoint])

Saving TensorBoard log files to: project1/softmax_101/20210718-085341
Epoch 1/3
2368/2368 [==============================] - 286s 105ms/step - loss: 2.3161 - accuracy: 0.4679 - val_loss: 1.2377 - val_accuracy: 0.6763
Epoch 2/3
2368/2368 [==============================] - 225s 94ms/step - loss: 1.3251 - accuracy: 0.6576 - val_loss: 1.1291 - val_accuracy: 0.6986
Epoch 3/3
2368/2368 [==============================] - 230s 94ms/step - loss: 1.1450 - accuracy: 0.7027 - val_loss: 1.0870 - val_accuracy: 0.7064


In [ ]:
evaluation = model.evaluate(test_data)
evaluation